In [12]:
from influxdb import InfluxDBClient

In [13]:
client = InfluxDBClient(host='localhost', port=8086)

In [14]:
client.create_database('eeg_test')

In [15]:
client.get_list_database()

[{'name': '_internal'}, {'name': 'test'}, {'name': 'eeg_test'}]

In [16]:
client.switch_database('eeg_test')

In [24]:
import random, datetime


def gen_rand_eeg():
    return {
        "Q": random.uniform(0, 1682.815),
        "R": random.uniform(0, 1682.815),
        "S": random.uniform(0, 1682.815),
        "T": random.uniform(0, 1682.815)
    }


def gen_rand_data():
    data = []
    timenow = datetime.datetime.now(datetime.timezone.utc)
    for i in range(1000):
        timenow += datetime.timedelta(seconds=1)
        data.append({
            'measurement': 'data',
            'tags': {
                'userID': 'steve'
            },
            'time': timenow.isoformat(),
            'fields': gen_rand_eeg()
        })
    return data


data = gen_rand_data()
data[0:2]

[{'fields': {'Q': 1062.5952144534256,
   'R': 1561.704870565966,
   'S': 44.45597824225809,
   'T': 217.6890686651401},
  'measurement': 'data',
  'tags': {'userID': 'steve'},
  'time': '2018-10-20T17:04:18.969406+00:00'},
 {'fields': {'Q': 1162.3777105257082,
   'R': 333.0040835411235,
   'S': 284.86553626553126,
   'T': 37.12272400618031},
  'measurement': 'data',
  'tags': {'userID': 'steve'},
  'time': '2018-10-20T17:04:19.969406+00:00'}]

In [25]:
client.write_points(data)

True

In [23]:
results = client.query('SELECT * FROM "eeg_test"."autogen"."data" WHERE time > now() - 10000s GROUP BY "user"')
results.raw

{'statement_id': 0}

In [23]:
points = results.get_points(tags={'user':'Carol'})

for point in points:
    print('time: {} duration: {}'.format(point['time'], point['duration']))

time: 2018-03-28T08:01:00Z duration: 127
time: 2018-03-29T08:04:00Z duration: 132
time: 2018-03-30T08:02:00Z duration: 129
